In [4]:
!pip install ta

In [5]:
import pandas as pd
import ta
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [6]:
def get_asset_history(ticker_symbol, start_date, end_date, interval):
  ticker_data = yf.Ticker(ticker_symbol)
  ticker_df = ticker_data.history(interval = interval, start = start_date, end = end_date)

  return ticker_df.Close

# Backtesting function



In [7]:
def backtest(close_prices, initial_capital):
  price_data_df = close_prices.to_frame()

  price_data_df["MACD"] = ta.trend.macd(close_prices, window_slow = 26, window_fast = 12)
  price_data_df["MACD_Signal"] = ta.trend.macd_signal(close_prices, window_slow = 26, window_fast = 12)
  price_data_df["RSI"] = ta.momentum.rsi(close_prices, 14)

  price_data_df["Buy_Signal"] = False
  price_data_df["Sell_Signal"] = False
  in_position = False
  price_data_df["Position"] = None
  price_data_df.loc[price_data_df.index[0], "Position"] = 0

  for date in range(0, len(price_data_df)):
    datestr = price_data_df.index[date].strftime("%Y-%m-%d")
    yesterdaystr = price_data_df.index[date - 1].strftime("%Y-%m-%d")

    if ((price_data_df.loc[datestr, "MACD"] > price_data_df.loc[datestr, "MACD_Signal"]) \
        or (price_data_df.loc[yesterdaystr, "MACD"] <= price_data_df.loc[yesterdaystr, "MACD_Signal"])) \
        and (price_data_df.loc[datestr, "RSI"] > 48) \
        and not in_position:

        price_data_df.loc[datestr, "Buy_Signal"] = True
        price_data_df.loc[datestr, "Position"] = 1
        in_position = True

    elif (price_data_df.loc[datestr, "MACD"] < price_data_df.loc[datestr, "MACD_Signal"]) \
        and (price_data_df.loc[yesterdaystr, "MACD"] >= price_data_df.loc[yesterdaystr, "MACD_Signal"]) \
        and in_position:

        price_data_df.loc[datestr, "Sell_Signal"] = True
        price_data_df.loc[datestr, "Position"] = 0
        in_position = False

  price_data_df["Position"] = price_data_df["Position"].ffill().fillna(0)
  price_data_df["Portfolio_Value"] = initial_capital
  price_data_df['Strategy_Return'] = price_data_df['Close'].pct_change() * price_data_df["Position"].shift(1)
  price_data_df['Portfolio_Value'] = (price_data_df['Strategy_Return'] + 1).cumprod()  * price_data_df["Portfolio_Value"].shift(1)
  price_data_df.loc[price_data_df.index[0], 'Portfolio_Value'] = initial_capital

  return price_data_df["Portfolio_Value"]

# Strategy evaluation metrics


In [8]:
def net_profit(portfolio_values):
  final_value = portfolio_values[-1]
  initial_value = portfolio_values[0]

  total_revenue = final_value - initial_value
  total_expenses = 0

  net_profit = total_revenue - total_expenses

  return net_profit



In [9]:
def max_drawdown(portfolio_values):
  values = pd.DataFrame(data = portfolio_values.cummax())
  values.columns = ["Cumulative_Max"]

  values['Drawdown'] = (portfolio_values - values['Cumulative_Max']) / values['Cumulative_Max']
  max_drawdown = values['Drawdown'].min()

  return max_drawdown

In [28]:
def drawdown_period(portfolio_values):
  values = pd.DataFrame(data = portfolio_values.cummax())
  values.columns = ["Cumulative_Max"]

  values['Drawdown'] = (portfolio_values - values['Cumulative_Max']) / values['Cumulative_Max']

  min_date = values['Drawdown'].idxmin()

  start_date = values.loc[:min_date, 'Cumulative_Max'].idxmax()
  after_min = values.loc[min_date:, 'Cumulative_Max']
  end_date = after_min[after_min == values.loc[start_date, "Cumulative_Max"]].index[0]

  drawdown_period = (end_date - start_date).days

  return drawdown_period

In [11]:
import math

In [12]:
def sharpe(portfolio_values, annual_interest_rate = 0.04):
  values = pd.DataFrame(data = portfolio_values.pct_change())
  values.columns = ["Daily_Return"]

  daily_risk_free_rate = (1 + annual_interest_rate) ** (1 / 365) - 1
  excess_daily_returns = values["Daily_Return"] - daily_risk_free_rate

  avg_excess_daily_return = values["Daily_Return"].mean()
  std_excess_daily_return = values["Daily_Return"].std()

  trading_days_per_year = 365
  annual_return = (1 + avg_excess_daily_return) ** trading_days_per_year - 1
  annual_std = std_excess_daily_return * math.sqrt(trading_days_per_year)

  sharpe_ratio = (annual_return) / annual_std

  return sharpe_ratio


In [13]:
def sortino(portfolio_values, annual_interest_rate = 0.04):
  values = portfolio_values.pct_change()
  values.columns = ["Daily_Return"]

  daily_risk_free_rate = (1 + annual_interest_rate) ** (1 / 365) - 1
  excess_daily_returns = values - daily_risk_free_rate

  avg_excess_daily_return = excess_daily_returns.mean()

  downside_returns = excess_daily_returns[excess_daily_returns < 0]
  downside_deviation = downside_returns.std()

  trading_days_per_year = 365
  annual_return = (1 + avg_excess_daily_return) ** trading_days_per_year - 1
  annual_downside_deviation = downside_deviation * math.sqrt(trading_days_per_year)

  sortino_ratio = (annual_return - annual_interest_rate) / annual_downside_deviation
  return sortino_ratio


# Perform backtest for every asset

In [14]:
def evaluate(symbol_name, portfolio_values):
  net_profit_value = net_profit(portfolio_values)
  max_drawdown_value = max_drawdown(portfolio_values)
  drawdown_period_value = drawdown_period(portfolio_values)
  sharpe_ratio = sharpe(portfolio_values)
  sortino_ratio = sortino(portfolio_values)

  print(f"for {symbol_name}:")
  print("- - - - - - - - - - - - - - - - - - - - - -")
  print(f"Net Profit: {net_profit_value}")
  print(f"Maximum Drawdown: {max_drawdown_value}")
  print(f"Drawdown Period: {drawdown_period_value}")
  print(f"Sharpe Ratio: {sharpe_ratio}")
  print(f"Sortino Ratio: {sortino_ratio}")
  print("===========================================\n")

In [15]:
ticker_symbols = {
    "btc": "BTC-USD",
    "eth": "ETH-USD",
    "doge": "DOGE-USD",
}
start_date = "2022-10-01"
end_date = "2024-10-01"
interval = "1d"
initial_capital = 100

In [16]:
btc_data = get_asset_history(ticker_symbols["btc"], start_date, end_date, interval)

In [17]:
btc_return = backtest(btc_data, initial_capital)

<ipython-input-7-bc5569e8cfcd>:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  price_data_df["Position"] = price_data_df["Position"].ffill().fillna(0)


In [18]:
eth_data = get_asset_history(ticker_symbols["eth"], start_date, end_date, interval)

In [19]:
eth_return = backtest(eth_data, initial_capital)

<ipython-input-7-bc5569e8cfcd>:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  price_data_df["Position"] = price_data_df["Position"].ffill().fillna(0)


In [20]:
doge_data = get_asset_history(ticker_symbols["doge"], start_date, end_date, interval)

In [21]:
doge_return = backtest(doge_data, initial_capital)

<ipython-input-7-bc5569e8cfcd>:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  price_data_df["Position"] = price_data_df["Position"].ffill().fillna(0)


# Evaluate the results

In [29]:
evaluate(ticker_symbols["btc"], btc_return)

evaluate(ticker_symbols["eth"], eth_return)

evaluate(ticker_symbols["doge"], doge_return)

for BTC-USD:
- - - - - - - - - - - - - - - - - - - - - -
Net Profit: 169.32782006906143
Maximum Drawdown: -0.3204312707600197
Drawdown Period: 145
Sharpe Ratio: 1.8087528012323477
Sortino Ratio: 2.3938834450720656

for ETH-USD:
- - - - - - - - - - - - - - - - - - - - - -
Net Profit: 62.15003350885527
Maximum Drawdown: -0.4209962878698351
Drawdown Period: 149
Sharpe Ratio: 0.879063211233554
Sortino Ratio: 0.9926060897204514

for DOGE-USD:
- - - - - - - - - - - - - - - - - - - - - -
Net Profit: -36.54510038458629
Maximum Drawdown: -0.6351003456668745
Drawdown Period: 169
Sharpe Ratio: 0.0585596260864178
Sortino Ratio: -0.07377950796220736



<ipython-input-8-1a589a4a07df>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  final_value = portfolio_values[-1]
<ipython-input-8-1a589a4a07df>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  initial_value = portfolio_values[0]
<ipython-input-8-1a589a4a07df>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  final_value = portfolio_values[-1]
<ipython-input-8-1a589a4a07df>:3: FutureWarning: Series.__getitem__ treating keys as posit